In [1]:
import pandas as pd
data = pd.read_csv('Load Data in kW.csv')
data.head()

,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,0:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
hour_1 = data.iloc[0:-1, 2]
hour_2 = data.iloc[1:, 2]

In [3]:
hour_2 = hour_2.reset_index()
hour_2 = hour_2['Load (kW)']

In [4]:
df = pd.concat([hour_1, hour_2], axis = 1)

In [17]:
df.columns = ['Hour_1', 'Hour_2']
df.head()

,Hour_1,Hour_2
0,5551.82208,4983.17184
1,4983.17184,4888.39680
2,4888.39680,5072.95872
3,5072.95872,5196.25980
4,5196.25980,5641.29720


In [18]:
normalized_df = (df - df.mean()) / df.std()
normalized_df.head()

,Hour_1,Hour_2
0,-0.446228,-0.979915
1,-0.979438,-1.068773
2,-1.068307,-0.895734
3,-0.895247,-0.780131
4,-0.779630,-0.362881


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(normalized_df.Hour_1, normalized_df.Hour_2, test_size = 0.10, random_state = 42)

In [8]:
m = 1 #Initial value of slope
c = -1 #Initial value of intercept
lr = 0.01 #Learning Rate
delta_m = 1 #Initialising Δm
delta_c = 1 #Initialising Δc
v_m = 0
v_c = 0
lam = 0.9
max_iters = 100 #Maximum number of iterations  
iters_count = 0 #Counting Iterations


def deriv(m_f, c_f, x, y, v1, v2):
  global lam
  m_deriv = -1 * (y - (m_f - lam * v1) * x - c_f + lam * v2) * x
  c_deriv = -1 * (y - (m_f - lam * v1) * x - c_f + lam * v2)
  return m_deriv, c_deriv  


while iters_count < max_iters:
  for i in range(x_train.shape[0]):
    delta_m, delta_c = deriv(m, c, x_train.iloc[i], y_train.iloc[i], v_m, v_c)
    v_m = lam * v_m - lr * delta_m
    v_c = lam * v_c - lr * delta_c
    m += v_m
    c += v_c
  iters_count += 1

In [19]:
print(f"\nThe local minima occurs at: {m}, {c}")


The local minima occurs at: 0.8497957222095366, -0.15662392198536862


In [20]:
import numpy as np

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [11]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m * i) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

In [12]:
y_pred_test = []
for i in x_test:
  y_p_te = (m * i) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

In [13]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
mse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.6518990110770351
Mean square error 0.4249723206432163
Mean absolute error 0.43781836275406955


In [14]:
#Testing Accuracies
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.5874494300215501
Mean square error 0.3450968328326441
Mean absolute error 0.4057943073140245


In [21]:
dec_1st = []
dec_1st.append(df.iloc[-1, 1])
for hour in range(24):
  normalised_pred = (dec_1st[-1] - df.mean()) / df.std()
  pred_load = m * normalised_pred + c
  pred_load = (pred_load * df.std()) + df.mean()
  dec_1st.append(pred_load)

In [22]:
hour = input("Enter an hour(0-23) of 1st December, 2018 to predict the load")
print(f"Predicted Load (kW) at {hour} hours on 1st December, 2018: {dec_1st[1 + int(hour)][0]}")

Enter an hour(0-23) of 1st December, 2018 to predict the load5
Predicted Load (kW) at 5 hours on 1st December, 2018: 5677.428516564374
